# Prompt Engineering, explored with Semantic Kernel and Azure OpenAI

To quickly get started, follow these steps:

1. Install [Polyglot notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) in VSCode.
2. [Create a new Azure OpenAI service (or use an existing OpenAI service)](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart?tabs=command-line&pivots=programming-language-studio#prerequisites).
3. [Deploy the `gpt-35-turbo` and `text-embeddings-ada-002` models](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#working-with-models).
4. [Create an Azure Cognitive Search instance and enable the Semantic Search capability](https://learn.microsoft.com/en-us/azure/search/semantic-search-overview#enable-semantic-search).
5. Copy the `.env.example` file from the parent folder to `dotnet/.env` and paste the corresponding values from the resources you provisioned in the earlier steps.
6. Click on `Run All`.


> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

# Background

## What is Prompt Engineering?
Prompt engineering is an iterative approach for crafting and refining prompts to enhance interactions with Large Language Models (LLMs). Mastery of prompt engineering is key to unlocking the full potential of LLMs in various applications. This has been pivotal in achieving advanced use cases in Microsoft Copilots.

This notebook serves as your go-to resource for effective prompt engineering techniques.

### Best Practices: Insights from Azure OpenAI

#### Be Specific and Descriptive
Craft your prompts to be both specific and descriptive to minimize ambiguity. Using analogies or metaphors can aid in making the prompts more understandable and relatable to the model.

#### Be Repetitive
- **Repeat**: Reiterate key instructions to ensure clarity and focus in the model's output.
- **Order Matters**: The sequence in which you present instructions can influence the model’s response due to its recency bias.

#### Space Efficiency
- **Token Limitations**: Be aware of the token limits for the model you are invoking.
- **Data Formats**: Opt for tabular formats over JSON for greater space efficiency.
- **White Space**: Use space judiciously, as each extra space counts as a token and can limit the model's performance.



### High Level Objectives
1. Include
	> Include instructions of requesting the model not to make up stuff but stay with facts.
1. Restrict
	> Restrict the output (e.g., choose from a confined list instead of generating free form strings)
1. Add
	> Add Chain of Thought style of instruction, "Solve the problem step by step."
1. Repeat
	> Repeat most important instructions in the prompt a couple of times.
1. Position
	> Position most important instructions in the last making use of latency effect.


## Let's Get Started

We will be utilizing **Semantic Kernel** to orchestrate interactions with the `gpt-35-turbo` model, which is deployed on **Azure OpenAI** for brevity. Alternatively, you can also use the **Azure OpenAI SDK** for model orchestration.


## Setup the Kernel to Interact with Azure OpenAI

Install dependencies and import libraries.

In [1]:
#r "nuget: dotenv.net"
dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

Installed Packages dotenv.net, 3.1.3

In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta6"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 1.0.0-beta6"

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta6 Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 1.0.0-beta6

In [3]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;

### Setup
Initialize the kernel using Azure OpenAI Chat completion model and memory store for RaG related prompting

In [4]:
var builder = new KernelBuilder();

builder
.WithAzureOpenAIChatCompletionService(
    env["AZURE_OPENAI_CHAT_MODEL"],
    env["AZURE_OPENAI_ENDPOINT"],
    env["AZURE_OPENAI_API_KEY"]
);

IKernel kernel = builder.Build();

In [5]:
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
var memoryBuilder = new MemoryBuilder();
memoryBuilder
    .WithAzureOpenAITextEmbeddingGenerationService(
        env["AZURE_OPENAI_EMBEDDING_MODEL"],
        env["AZURE_OPENAI_ENDPOINT"],
        env["AZURE_OPENAI_API_KEY"]
    )

    .WithMemoryStore(new VolatileMemoryStore());

var memory = memoryBuilder.Build();


## Start with clear instructions
> Tell the model the task you want it to do at the beginning of the prompt and repeat at the end 

It’s obvious that the we want to provide the model with clear instructions rather than vague instruction but in addition the sequence in which information is fed in the prompt, matters (GPT style models are built in a certain way and that’s the reason behind this). Our research suggests that telling the model the task you want it to do at the beginning of the prompt, before sharing additional contextual information or examples, can help produce higher-quality outputs.

In this example, we give the exact statement we want to check (“Several sources mention a chance of another eruption”) before we give the snippet context. This allows the token representations extracted from the snippets to be tailored to the statement we are checking. And the resulting model response is accurate.

Models can be susceptible to “recency bias,” which means that information at the end of the prompt might have more significant influence over the output than information at the beginning of the prompt. Another technique is to try repeating the instructions at the end of the prompt.
Repeating the instruction at the beginning as well as the end of the prompt leads to a higher likelihood of getting an accurate model response.

In [6]:
string skPrompt = """
Your task is to verify if the statement "Several sources mention a chance of another large eruption" is supported by a specific quote from the following set of snippets.
{{$input}}
Is the statement "Several sources mention a chance of another large eruption" directly implied or stated by the snippets?
""";

var clearInstructionsFunction = kernel.CreateSemanticFunction(skPrompt, requestSettings: new OpenAIRequestSettings { MaxTokens = 2000, Temperature = 0.0, TopP = 0.5 });
string input = 
        """
        ---
        SNIPPETS
        [1] 14 percent chance of megaquake hitting Seattle, experts say
        SEATTLE - There's a 14 percent chance of a magnitude 9 Cascadia earthquake hitting Seattle in the next 50 years, the U.S. Geological Survey estimates. "Unfortunately, we are unable to...
        [2] Earthquake experts lay out latest outlook for Seattle's 'Really Big One’
        “We say that there's approximately a 14% chance of another approximately magnitude-9 earthquake occurring in the next 50 years,” said Erin Wirth, a geophysicist at the University of Washington...
        ---
        """;
var result = await kernel.RunAsync(input, clearInstructionsFunction);

Console.WriteLine(result.GetValue<string>());

No, the statement "Several sources mention a chance of another large eruption" is not directly implied or stated by the snippets. The snippets provided discuss the probability of a large earthquake, not an eruption. An eruption typically refers to a volcanic event, which is different from an earthquake.


## Prime the output
> Add phrases at the end of the prompt to obtain a model response in a desired form

Another technique is to prime the output, that is add phrases at the end of the prompt to obtain a model response in a desired format.
For example, as shown here for this particular task, it is important to "commit" the model to the list generation process
This refers to including a few words or phrases at the end of the prompt to obtain a model response that follows the desired form. For example, using a cue such as “Here’s a bulleted list of key points:\n-” can help make sure the outputted is formatted as a list of bullet points.

In the above prompt, the text "One possible search query is:" primes the model to produce an output in the form of a search query. This technique can help remove hedges that the model might typically add, such as “One possibility is...”.

In [7]:
string skPrompt = """
The future of artificial intelligence is bright. With Microsoft OpenAI, we are unlocking the potential of AI to help people achieve more. We are creating a platform that enables developers to build intelligent applications and services that can help people in their everyday lives. Our mission is to democratize AI so that everyone can benefit from its power. We are committed to advancing the state of the art in AI and making it accessible to everyone. With Microsoft OpenAI, we are taking the first steps towards a future where AI can be used to solve some of the world's most pressing challenges.

{{$input}}
""";

var primeOutputFunction = kernel.CreateSemanticFunction(skPrompt, requestSettings: new OpenAIRequestSettings { MaxTokens = 2000, Temperature = 0.0, TopP = 0.5 });
string input = 
        """
        Here’s a bulleted list of key points:\n-
        """;
var result = await kernel.RunAsync(input, primeOutputFunction);

Console.WriteLine(result.GetValue<string>());

- Microsoft OpenAI is focused on unlocking the potential of AI to help people achieve more.
- The platform enables developers to build intelligent applications and services.
- The mission is to democratize AI, making it accessible and beneficial to everyone.
- Microsoft OpenAI is committed to advancing the state of the art in AI.
- The vision is a future where AI can be used to solve some of the world's most pressing challenges.


In [8]:
string skPrompt = """
John Smith is married to Lucy Smith. They have five kids, and he works as a software engineer at Microsoft. 


What search queries should I do to fact-check this?

##

{{$input}}
""";

var func2 = kernel.CreateSemanticFunction(skPrompt, requestSettings: new OpenAIRequestSettings { MaxTokens = 2000, Temperature = 0.0, TopP = 0.5 });
string input = 
        """
        One possible search query is:
        """;
var result = await kernel.RunAsync(input, func2);

Console.WriteLine(result.GetValue<string>());

"John Smith Microsoft software engineer"
Another possible search query could be: 
"John Smith Lucy Smith marriage"
And another one could be:
"John Smith five children"

Please note that "John Smith" is a very common name, so you may need to add more specific details to your search if you have them, such as a location or a more specific job title. Also, personal information like this may not be publicly available or easily found online due to privacy reasons.


## Add clear syntax
> Include punctuation, headings, and section markers to help communicate intent

Using clear syntax for your prompt—including punctuation, headings, and section markers—helps communicate intent and often makes outputs easier to parse.
In the example below, separators (“---” in this case) have been added between different sources of information or steps. This allows the use of “---” as a stopping condition for generation. In addition, section headings or special variables are presented in uppercase to differentiate them.

If you’re not sure what syntax to use, consider using markdown or XML, since LLMs have been trained on a lot of web content in XML or markdown

In [9]:
string skPrompt = """
You will read a paragraph, and then issue queries to a search engine in order to fact-check it. Also explain the queries.

---
PARAGRAPH

{{$input}}

---
""";

var clearSyntax = kernel.CreateSemanticFunction(skPrompt, requestSettings: new OpenAIRequestSettings { MaxTokens = 2000, Temperature = 0.0, TopP = 0.5 });
string input = 
        """
        John Smith is married to Lucy Smith. They have five kids, and he works as a software engineer at Microsoft. What search queries should I do to fact-check this?
        """;
var result = await kernel.RunAsync(input, clearSyntax);

Console.WriteLine(result.GetValue<string>());

To fact-check the information provided in the paragraph, you could use the following search queries:

1. "John Smith software engineer Microsoft": This query is to verify if John Smith is indeed a software engineer at Microsoft. However, it's worth noting that "John Smith" is a very common name, so you may need additional information to confirm this fact.

2. "John Smith Lucy Smith marriage": This query is to confirm if John Smith is married to Lucy Smith. Again, due to the commonality of these names, additional information might be needed to confirm this fact.

3. "John Smith Lucy Smith children": This query is to verify if John Smith and Lucy Smith have five children. 

Please note that these queries are based on the assumption that John Smith is a public figure or has a public presence. If he is a private individual, it's unlikely that this information would be readily available due to privacy laws and regulations.


## Prompt Chaining

Prompt chaining is a technique that involves concatenating several prompts to create a more complex prompt that directs the model towards specific outputs. This technique is powerful for prompt engineering because it enables the creation of a more focused prompt that provides the model with more specific information to generate an output. By chaining prompts, the model is directed to focus on specific aspects of the task or problem at hand, improving the accuracy and relevance of the generated outputs. Additionally, prompt chaining allows for the integration of diverse information sources, helping to ground the model in factual information and generate more accurate and relevant outputs. Overall, prompt chaining is a powerful technique for prompt engineering as it provides a flexible and customizable approach for directing models towards specific outputs, improving their effectiveness in a wide range of applications.

> Note: Use Semantic Kernel's planner (sequential or stepwise) to chain this properly. The approach below is used to convey the technique in the simplest possible way.

#### Entity Extraction:

In [13]:
string skPrompt = """
Please extract entities from the following news article: 
---
'{{$input}}'
---

Only reply entities and DO NOT add anything extra

""";

var summaryFunction = kernel.CreateSemanticFunction(skPrompt, requestSettings: new OpenAIRequestSettings { MaxTokens = 2000, Temperature = 0.0, TopP = 0.5 });
string input = 
        """
        The new iPhone model is set to be released next month. It has been highly anticipated by Apple fans and is expected to feature a larger screen and improved camera
        """;
var extractEntities = await kernel.RunAsync(input, summaryFunction);
var extractedEntities = extractEntities.GetValue<string>();

Console.WriteLine(extractedEntities);

iPhone, next month, Apple, larger screen, improved camera


#### Chained with Summarization

In [15]:
string skPrompt = """
Please summarize the information about the product:

{{$input}}

""";

var summaryFunction = kernel.CreateSemanticFunction(skPrompt, requestSettings: new OpenAIRequestSettings { MaxTokens = 2000, Temperature = 0.0, TopP = 0.5 });
var summary = await kernel.RunAsync(extractedEntities, summaryFunction);
var summaryOutput = summary.GetValue<string>();

Console.WriteLine(summaryOutput);

Next month, Apple is set to release a new iPhone featuring a larger screen and an improved camera.


#### Chained with Sentiment Analysis

In [17]:
string skPrompt = """
Please provide a sentiment for the following text:

---
{{$input}}
---

Ony respond in one word: positive, negative, or neutral

""";

var summaryFunction = kernel.CreateSemanticFunction(skPrompt, requestSettings: new OpenAIRequestSettings { MaxTokens = 2000, Temperature = 0.0, TopP = 0.5 });
var sentimentFunc = await kernel.RunAsync(summaryOutput, summaryFunction);

Console.WriteLine( sentimentFunc.GetValue<string>());

Positive


## Few-shot learning
> Also known as in-context learning, it allows the model to interact with new knowledge

We talked about few-shot before but again it is a very common way to adapt language models to new tasks is to use few-shot learning, also known as in-context learning. In few-shot learning a set of training examples is provided in the prompt and then the LLM is asked to complete one more unfinished example.
In the following example we use an instruction combined with few-shot learning make up puns
Choose your few-shot examples carefully and ensure they cover a variety of circumstances relevant to your scenario, including edge cases.
Also, as shared earlier, the LLM models can exhibit a form of recency bias. This means that the order in which "few-shot" examples are provided to the model, matters. If you don't want this to matter, consider sampling multiple completions from prompts based on randomized orderings of the examples or list items.


In [19]:
string skPrompt = """
Write a list of puns.

---
{{$input}}
""";

var clearSyntax = kernel.CreateSemanticFunction(skPrompt, requestSettings: new OpenAIRequestSettings { MaxTokens = 2000, Temperature = 0.0, TopP = 0.5 });
string fewShotExamples = 
        """
        1. "Why did Adele cross the road? To say hello from the other side."

        2. "What kind of concert only costs 45 cents? A 50 Cent concert featuring Nickelback."

        3. "What did the grape say when it got crushed? Nothing, it just let out a little wine."

        4. "What was Forrest Gump's email password? 1forrest1"

        5. "Can February March? No, but April May."

        6. "What do you call fancy language model?

        """;
var result = await kernel.RunAsync(fewShotExamples, clearSyntax);

Console.WriteLine(result.GetValue<string>());

"A sophisticated AI-ristocrat."


## Few-Shot Reasoning

Few-shot reasoning is similar to few-shot prompt but instead here we provide an reasoning example for a specific task to help the model reason on a different task. For example here, if ask the model to answer a simple logical reasoning task, it fails, but if we provide the model with a similar logical reasoning example , it shows the model how it can solve for a different similar logical task. Here in the example on the left , we provide the prompt with a different example that has the same logical reasoning needed to solve the problem and as a result the model understand the mechanism needed to solve for the new problem. 



In [20]:
string skPrompt = """
Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

Answer: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5+6 = 11. The answer is 11. 

---
{{$input}}
""";

var clearSyntax = kernel.CreateSemanticFunction(skPrompt, requestSettings: new OpenAIRequestSettings { MaxTokens = 2000, Temperature = 0.0, TopP = 0.5 });
string fewShotExamples = 
        """
        The cafeteria has 23 apples. If they used 20 to make lunch and bought 6 more, how many do they have?

        """;
var result = await kernel.RunAsync(fewShotExamples, clearSyntax);

Console.WriteLine(result.GetValue<string>());

The cafeteria started with 23 apples. They used 20, so they had 23 - 20 = 3 apples left. Then they bought 6 more, so 3 + 6 = 9. The cafeteria now has 9 apples.


## Break the task down
> LLMs often perform better if the task is broken down into smaller step

LLMs often perform better if the task is broken down into smaller steps. For example, in the search query generation prompt referenced earlier, the prompt can be restructured so that the model is first instructed to extract relevant facts, and then instructed to generate search queries that can be used to verify those facts.

Notice the use of clear syntax to differentiate the sections and prime the output. In this simple example, breaking the task down from one to two steps is not very dramatic, but when trying to do this for a larger piece of text with many factual claims, breaking the task down can make a significant difference.

In [22]:
string skPrompt = """
You will read a paragraph, and then issue queries to a search engine in order to fact-check it. 
--- 

PARAGRAPH 

John Smith is married to Lucy Smith. They have five kids, and he works as a software engineer at Microsoft. What search queries should I do to fact-check this? 
--- 

{{$input}}

FACTUAL CLAIMS

""";

var clearSyntax = kernel.CreateSemanticFunction(skPrompt, requestSettings: new OpenAIRequestSettings { MaxTokens = 2000, Temperature = 0.0, TopP = 0.5 });
string fewShotExamples = 
        """
        Now you will extract factual claims first, and then issue queries to fact-check them. When issuing a query, use the function SEARCH("query") 
        """;
var result = await kernel.RunAsync(fewShotExamples, clearSyntax);

Console.WriteLine(result.GetValue<string>());

1. John Smith is married to Lucy Smith.
2. They have five kids.
3. He works as a software engineer at Microsoft.

SEARCH QUERIES

1. SEARCH("John Smith married to Lucy Smith")
2. SEARCH("John and Lucy Smith number of children")
3. SEARCH("John Smith software engineer at Microsoft")


## Guardrails
> Provide specific instructions to limit and context to the output of completion 

To demonstrate this further, you can use meta-prompts / system prompts to provide guardrails to the models output. In this example here , if we ask the model what is cosmos, it replies that cosmos is open-source blockchain decentralized network. This is not the answer that we wanted. If we provide a system message in addition to the prompt, telling the model that we only want specific information on Microsoft products, we will get the answer we desire. The important thing to note here is that we are also telling the model not to discuss other topics that are not related to Microsoft. This is important to reduce hullicanations and provide the model guardrails on what it should not respond. 